In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 85% 73.0M/85.7M [00:00<00:00, 64.2MB/s]
100% 85.7M/85.7M [00:01<00:00, 88.4MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 8.1 MB/s 
     |████████████████████████████████| 87 kB 9.7 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 1.2 MB 87.3 MB/s 
     |████████████████████████████████| 234 kB 81.6 MB/s 
     |████████████████████████████████| 77 kB 7.8 MB/s 
     |████████████████████████████████| 6.4 MB 64.6 MB/s 
     |████████████████████████████████| 840 kB 90.1 MB/s 
     |████████████████████████████████| 3.4 MB 76.5 MB/s 
     |████████████████████████████████| 120 kB 56.8 MB/s 
     |████████████████████████████████| 1.1 MB 87.1 MB/s 
     |████████████████████████████████| 1.1 MB 84.4 MB/s 
     |████████████████████████████████| 25.3 MB 87.2 MB/s 
     |████████████████████████████████| 352 kB 90.8 MB/s 
     |████████████████████████████████| 99 kB 12.6 MB/s 
     |████████████████████████████████| 47.7 MB 116 kB/s 
     |████████████████████████████████| 462 kB 83.1 MB/s 
     |█████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [10]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [11]:
spec = model_spec.get('efficientdet_lite3')

In [12]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=30, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/30
397/397 [==============================] - 250s 500ms/step - det_loss: 0.9517 - cls_loss: 0.7482 - box_loss: 0.0041 - reg_l2_loss: 0.0938 - loss: 1.0454 - learning_rate: 0.0078 - gradient_norm: 3.6133 - val_det_loss: 0.5846 - val_cls_loss: 0.5344 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.0941 - val_loss: 0.6787
Epoch 2/30
397/397 [==============================] - 196s 495ms/step - det_loss: 0.6191 - cls_loss: 0.5089 - box_loss: 0.0022 - reg_l2_loss: 0.0944 - loss: 0.7135 - learning_rate: 0.0074 - gradient_norm: 3.6802 - val_det_loss: 0.4650 - val_cls_loss: 0.4218 - val_box_loss: 8.6286e-04 - val_reg_l2_loss: 0.0946 - val_loss: 0.5596
Epoch 3/30
397/397 [==============================] - 196s 493ms/step - det_loss: 0.5530 - cls_loss: 0.4660 - box_loss: 0.0017 - reg_l2_loss: 0.0948 - loss: 0.6478 - learning_rate: 0.0074 - gradient_norm: 3.0108 - val_det_loss: 0.4078 - val_cls_loss: 0.3652 - val_box_loss: 8.5301e-04 - val_reg_l2_loss: 0.0949 - val_loss: 0.5027
Epoch 4/30
397

In [13]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 6080.623876571655


In [14]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 11s 103ms/step

{'AP': 0.831334, 'AP50': 0.9298945, 'AP75': 0.8881959, 'APs': -1.0, 'APm': 0.54478085, 'APl': 0.8472001, 'ARmax1': 0.7592451, 'ARmax10': 0.8764373, 'ARmax100': 0.8826016, 'ARs': -1.0, 'ARm': 0.6547619, 'ARl': 0.8955642, 'AP_/10': 0.7960171, 'AP_/20': 0.8460947, 'AP_/50': 0.8361172, 'AP_/100': 0.9368319, 'AP_/200': 0.8680846, 'AP_/500': 0.81500536, 'AP_/2000': 0.72118694}


In [15]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 14.847809076309204


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [16]:
model.export(export_dir='/content/')